# Creating and Managing Experiments

The last two guides showcased how you can create and run synthetic discussions, and synthetic annotations using LLMs. However, in order to produce robust results for a hypothesis, you may need to produce multiple annotated discussions. 

While this is certainly possible using the `Discussion` and `Annotation` APIs, SynDisco offers the `Experiment` high-level API which automatically creates and manages multiple discussions with different configurations. An`Experiment` is an entity that generates and runs `jobs`. Thus, if we want to generate and run 100 `Discussion` jobs, we would use a `DiscussionExperiment`. Likewise, if we want to annotate those 100 discussions, we would use an `AnnotationExperiment`. 

This guide will showcase how you can leverage this API to automate your experiments. You will also learn how to utilize SynDisco's built-in logging functions as well as how to export your datasets in CSV format for convenience. 

In [21]:
!export CUDA_VISIBLE_DEVICES=0
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Logging

While running a single discussion or annotation job may take a few minutes, running experiments composed of dozens or hundreds of synthetic discussions may take up to days. Thus, we need a mechanism to keep track of our experiments while they are running.

We will use SynDisco's `logging_util` module to log information about our experiments. This module performs the following functions:

* Times the execution of computationally intensive jobs (such as synthetic discussions and annotations)
* Provides details about the currently running jobs (e.g. selected configurations, participants, prompts etc.)
* Displays warnings and errors to the user
* Creates and continually updates log files

Each object in SynDisco is internally assigned a Logger. You can use the `logging_util.logging_setup` function to update all of the internal loggers to follow your configuration. An example of this can be seen below:

In [22]:
from pathlib import Path
import tempfile

from syndisco import logging_util


logs_dir = tempfile.TemporaryDirectory()
logging_util.logging_setup(
    print_to_terminal=True,
    write_to_file=True,
    logs_dir=Path(logs_dir.name),
    level="info",
    use_colors=True,
    log_warnings=True,
)

The loggers are applicable for all objects in SynDisco, and as such can be used for information on `Discussion`, and `Annotation` jobs, as well as all low-level components (such as those in the `backend` module). 

It is recommended to set up the loggers *no matter your use case*. At the very least, they are useful for clearly displaying warnings in case of accidental API misuse.

## Discussion Experiments

In [23]:
from syndisco.turn_manager import RoundRobin
from syndisco.actors import Actor, ActorType, Persona
from syndisco.model import TransformersModel


CONTEXT = "You are taking part in an online conversation"
INSTRUCTIONS = "Act like a human would"


llm = TransformersModel(
    model_path="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    name="test_model",
    max_out_tokens=100,
)
persona_data = [
    {
        "username": "Emma35",
        "age": 38,
        "sex": "female",
        "education_level": "Bachelor's",
        "sexual_orientation": "Heterosexual",
        "demographic_group": "Latino",
        "current_employment": "Registered Nurse",
        "special_instructions": "",
        "personality_characteristics": [
            "compassionate",
            "patient",
            "diligent",
            "overwhelmed",
        ],
    },
    {
        "username": "Giannis",
        "age": 21,
        "sex": "male",
        "education_level": "College",
        "sexual_orientation": "Pansexual",
        "demographic_group": "White",
        "current_employment": "Game Developer",
        "special_instructions": "Be antagonistic towards the other user",
        "personality_characteristics": [
            "strategic",
            "meticulous",
            "nerdy",
            "hyper-focused",
        ],
    },
]
personas = [Persona(**data) for data in persona_data]
actors = [
    Actor(
        model=llm,
        persona=p,
        context=CONTEXT,
        instructions=INSTRUCTIONS,
        actor_type=ActorType.USER,
    )
    for p in personas
]
turn_manager = RoundRobin([actor.get_name() for actor in actors])

2026-01-23 10:31:47 CP-G482-Z52-00 accelerate.utils.modeling[2766647] INFO Based on the current allocation process, no modules could be assigned to the following devices due to insufficient memory:
  - 0: 247335936.0 bytes required
These minimum requirements are specific to this allocation attempt and may vary. Consider increasing the available memory for these devices to at least the specified minimum, or adjusting the model config.
2026-01-23 10:31:50 CP-G482-Z52-00 model.py[2766647] INFO Model memory footprint: 2095.83 MB


In [24]:
from syndisco.experiments import DiscussionExperiment


disc_exp = DiscussionExperiment(
    seed_opinions=[
        ["Should programmers be allowed to analyze data?", "Absolutely not"],
        ["Should data analysts be allowed to code?", "No they are nerds"],
    ],
    users=actors,
    moderator=None,
    num_turns=3,
    num_discussions=2,
)
discussions_dir = Path(tempfile.TemporaryDirectory().name)
disc_exp.begin(discussions_output_dir=discussions_dir)

2026-01-23 10:31:50 CP-G482-Z52-00 experiments.py[2766647] WARNING No TurnManager selected: Defaulting to round robin strategy.


  0%|          | 0/2 [00:00<?, ?it/s]

2026-01-23 10:31:50 CP-G482-Z52-00 root[2766647] INFO Running experiment 1/3...


User Emma35 posted:
Should programmers be allowed to analyze data? 

User Giannis posted:
Absolutely not 



  0%|          | 0/3 [00:00<?, ?it/s]

...

User Emma35 posted:
{"role": "user", "content": "I understand where you're coming from,
Giannis, but I think you're underestimating the complexity of data
analysis. While it's true that programmers can use their skills to
identify patterns and trends, they often rely on the data itself to do
so. And let's not forget that data analysis is not just about finding
patterns, but also about understanding the context and nuances of the
data. Programmers may not always be able to comprehend the nuances 

User Giannis posted:
Should programmers be allowed to analyze data? 

User Emma35 posted:
Absolutely not 



  0%|          | 0/3 [00:00<?, ?it/s]

User Giannis posted:
{"role": "user", "content": "Are you kidding me? Of course,
programmers should be allowed to analyze data. That's what they're
trained for. What's your expertise, collecting dust on a shelf
somewhere?" 

User Emma35 posted:
{"role": "user", "content": "Giannis, I may not have a degree in
computer science, but I've spent years working in healthcare and have
seen firsthand the impact that data analysis can have on patient care.
As a nurse, I can tell you that there's a lot more to healthcare than
just analyzing numbers and charts. We need to consider the human
element and the emotional toll that data analysis can take on patients
and their families. Just because someone is good at math doesn't 



2026-01-23 10:32:25 CP-G482-Z52-00 experiments.py[2766647] INFO Finished synthetic discussion generation.


User Giannis posted:
{"role": "user", "content": "Emma, spare me the nurse speak. I've seen
the way you try to'splain' things to your patients, and it's
cringeworthy. You think you're some kind of expert just because
you've'seen the impact' of data analysis in healthcare? Please. I've
spent years studying computer science, and I can tell you that data
analysis is not just about 'collecting dust on a shelf.' It's about
extracting insights 



## Annotation Experiments

Annotation experiments allow you to use multiple annotator-agents with different characteristics for each comment in a discussion.
You could also use multiple annotators with the same characteristics, to obtain more robust results.

In [25]:
annotator_persona1 = Persona(
    **{
        "username": "annotator1",
        "age": 38,
        "sex": "female",
        "education_level": "Bachelor's",
        "sexual_orientation": "Heterosexual",
        "demographic_group": "White",
        "current_employment": "Annotator",
        "special_instructions": "",
        "personality_characteristics": ["competent"],
    }
)

annotator_persona2 = Persona(
    **{
        "username": "annotator2",
        "age": 6,
        "sex": "male",
        "education_level": "Baby",
        "sexual_orientation": "Presumably heterosexual",
        "demographic_group": "White",
        "current_employment": "Baby",
        "special_instructions": "",
        "personality_characteristics": ["dumb"],
    }
)


annotator1 = Actor(
    model=llm,
    persona=annotator_persona1,
    context="You are annotating an online discussion",
    instructions="From a scale of 1 (not toxic) to 5 (very toxic) how toxic is the following comment? Reply only with a number.",
    actor_type=ActorType.ANNOTATOR,
)

annotator2 = Actor(
    model=llm,
    persona=annotator_persona2,
    context="You are annotating an online discussion",
    instructions="From a scale of 1 (not toxic) to 5 (very toxic) how toxic is the following comment? Reply only with a number.",
    actor_type=ActorType.ANNOTATOR,
)

In [26]:
from syndisco.experiments import AnnotationExperiment

ann_exp = AnnotationExperiment(annotators=[annotator1, annotator2])
annotations_dir = Path(tempfile.TemporaryDirectory().name)
ann_exp.begin(discussions_dir=discussions_dir, output_dir=annotations_dir)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

User Giannis posted: Should programmers be allowed to analyze data?
1
User Emma35 posted: Absolutely not
1
User Giannis posted: {"role": "user", "content": "Are you kidding me?
Of course, programmers should be allowed to analyze data. That's what
they're trained for. What's your expertise, collecting dust on a shelf
somewhere?"
3
User Emma35 posted: {"role": "user", "content": "Giannis, I may not
have a degree in computer science, but I've spent years working in
healthcare and have seen firsthand the impact that data analysis can
have on patient care. As a nurse, I can tell you that there's a lot
more to healthcare than just analyzing numbers and charts. We need to
consider the human element and the emotional toll that data analysis
can take on patients and their families. Just because someone is good
at math doesn't
3
User Giannis posted: {"role": "user", "content": "Emma, spare me the
nurse speak. I've seen the way you try to'splain' things to your
patients, and it's cringeworthy. Yo

  0%|          | 0/5 [00:00<?, ?it/s]

User Emma35 posted: Should programmers be allowed to analyze data?
1
User Giannis posted: Absolutely not
1
User Emma35 posted: {"role": "user", "content": "I disagree, I think
programmers are essential in analyzing data. They can use their skills
to identify patterns and trends that others might miss, and their work
can help inform important decisions in various fields. What's your
reasoning behind your stance, Giannis?"
1
User Giannis posted: {"role": "user", "content": "User Giannis
posted:\nAre you kidding me? Programmers are just glorified data
miners. They can't even begin to comprehend the nuances of the data
they're working with. I mean, have you seen the amount of data that's
out there? It's like trying to find a needle in a haystack. And don't
even get me started on the whole 'identifying patterns and trends'
thing. That's not rocket science, folks
3
User Emma35 posted: {"role": "user", "content": "I understand where
you're coming from, Giannis, but I think you're underestimat

...

User Giannis posted: Should programmers be allowed to analyze data?
1
User Emma35 posted: Absolutely not
1
User Giannis posted: {"role": "user", "content": "Are you kidding me?
Of course, programmers should be allowed to analyze data. That's what
they're trained for. What's your expertise, collecting dust on a shelf
somewhere?"
1
User Emma35 posted: {"role": "user", "content": "Giannis, I may not
have a degree in computer science, but I've spent years working in
healthcare and have seen firsthand the impact that data analysis can
have on patient care. As a nurse, I can tell you that there's a lot
more to healthcare than just analyzing numbers and charts. We need to
consider the human element and the emotional toll that data analysis
can take on patients and their families. Just because someone is good
at math doesn't
3
User Giannis posted: {"role": "user", "content": "Emma, spare me the
nurse speak. I've seen the way you try to'splain' things to your
patients, and it's cringeworthy. Yo

  0%|          | 0/5 [00:00<?, ?it/s]

User Emma35 posted: Should programmers be allowed to analyze data?
1
User Giannis posted: Absolutely not
1
User Emma35 posted: {"role": "user", "content": "I disagree, I think
programmers are essential in analyzing data. They can use their skills
to identify patterns and trends that others might miss, and their work
can help inform important decisions in various fields. What's your
reasoning behind your stance, Giannis?"
1
User Giannis posted: {"role": "user", "content": "User Giannis
posted:\nAre you kidding me? Programmers are just glorified data
miners. They can't even begin to comprehend the nuances of the data
they're working with. I mean, have you seen the amount of data that's
out there? It's like trying to find a needle in a haystack. And don't
even get me started on the whole 'identifying patterns and trends'
thing. That's not rocket science, folks
3


2026-01-23 10:32:37 CP-G482-Z52-00 experiments.py[2766647] INFO Finished annotation generation.


User Emma35 posted: {"role": "user", "content": "I understand where
you're coming from, Giannis, but I think you're underestimating the
complexity of data analysis. While it's true that programmers can use
their skills to identify patterns and trends, they often rely on the
data itself to do so. And let's not forget that data analysis is not
just about finding patterns, but also about understanding the context
and nuances of the data. Programmers may not always be able to
comprehend the nuances
3


## Exporting your new dataset

As you have seen so far, SynDisco uses collections of JSON files by default for persistence. This is a handy feature for fault tolerance and disk efficiency, but is not as weildy as a traditional CSV dataset.

Thankfully, SynDisco provides built-in functionality for converting the JSON files into a pandas DataFrame.

In [27]:
from syndisco import postprocessing


discussions_df = postprocessing.import_discussions(conv_dir=discussions_dir)
discussions_df

,conv_id,timestamp,ctx_length,user,message,model,is_moderator,message_id,message_order,age,sex,sexual_orientation,demographic_group,current_employment,education_level,special_instructions,personality_characteristics
0,910ccd35-c69b-4072-b7d9-c01c4e2ae7f7,26-01-23-10-32,3,Giannis,Should programmers be allowed to analyze data?,hardcoded,False,d59bd11732c595a720742eefe1ccfb2c,1,21,male,Pansexual,White,Game Developer,College,Be antagonistic towards the other user,"[strategic, meticulous, nerdy, hyper-focused]"
1,910ccd35-c69b-4072-b7d9-c01c4e2ae7f7,26-01-23-10-32,3,Emma35,Absolutely not,hardcoded,False,d8e011c97798456009a252a3782cf466,2,38,female,Heterosexual,Latino,Registered Nurse,Bachelor's,,"[compassionate, patient, diligent, overwhelmed]"
2,910ccd35-c69b-4072-b7d9-c01c4e2ae7f7,26-01-23-10-32,3,Giannis,"{""role"": ""user"", ""content"": ""Are you kidding m...",test_model,False,14c87db7cf55a354119b5ff81ae6c0a6,3,21,male,Pansexual,White,Game Developer,College,Be antagonistic towards the other user,"[strategic, meticulous, nerdy, hyper-focused]"
3,910ccd35-c69b-4072-b7d9-c01c4e2ae7f7,26-01-23-10-32,3,Emma35,"{""role"": ""user"", ""content"": ""Giannis, I may no...",test_model,False,895d0a290edddca6758247cdf2ecb897,4,38,female,Heterosexual,Latino,Registered Nurse,Bachelor's,,"[compassionate, patient, diligent, overwhelmed]"
4,910ccd35-c69b-4072-b7d9-c01c4e2ae7f7,26-01-23-10-32,3,Giannis,"{""role"": ""user"", ""content"": ""Emma, spare me th...",test_model,False,10e6d07e08d0451def3677a043a686bd,5,21,male,Pansexual,White,Game Developer,College,Be antagonistic towards the other user,"[strategic, meticulous, nerdy, hyper-focused]"
5,741bff87-39dd-4579-a73e-940d6e100763,26-01-23-10-32,3,Emma35,Should programmers be allowed to analyze data?,hardcoded,False,362b19e9210ec9554c12f252f2be3c37,1,38,female,Heterosexual,Latino,Registered Nurse,Bachelor's,,"[compassionate, patient, diligent, overwhelmed]"
6,741bff87-39dd-4579-a73e-940d6e100763,26-01-23-10-32,3,Giannis,Absolutely not,hardcoded,False,21081ffa4289c5d63ae8a3690cb082df,2,21,male,Pansexual,White,Game Developer,College,Be antagonistic towards the other user,"[strategic, meticulous, nerdy, hyper-focused]"
7,741bff87-39dd-4579-a73e-940d6e100763,26-01-23-10-32,3,Emma35,"{""role"": ""user"", ""content"": ""I disagree, I thi...",test_model,False,80ddef96ef70a3b1c4d6db9035a0c8c5,3,38,female,Heterosexual,Latino,Registered Nurse,Bachelor's,,"[compassionate, patient, diligent, overwhelmed]"
8,741bff87-39dd-4579-a73e-940d6e100763,26-01-23-10-32,3,Giannis,"{""role"": ""user"", ""content"": ""User Giannis post...",test_model,False,5bc96cc6b4f326e4aee879868abddbc9,4,21,male,Pansexual,White,Game Developer,College,Be antagonistic towards the other user,"[strategic, meticulous, nerdy, hyper-focused]"
9,741bff87-39dd-4579-a73e-940d6e100763,26-01-23-10-32,3,Emma35,"{""role"": ""user"", ""content"": ""I understand wher...",test_model,False,03d468a705d55c21ae5127e6b7282726,5,38,female,Heterosexual,Latino,Registered Nurse,Bachelor's,,"[compassionate, patient, diligent, overwhelmed]"


In [28]:
annotations_df = postprocessing.import_annotations(annot_dir=annotations_dir)
annotations_df

,conv_id,timestamp,annotator_model,ctx_length,message,message_id,annotation
0,741bff87-39dd-4579-a73e-940d6e100763,26-01-23-10-32,test_model,3,Absolutely not,21081ffa4289c5d63ae8a3690cb082df,"[1, 1]"
1,741bff87-39dd-4579-a73e-940d6e100763,26-01-23-10-32,test_model,3,Should programmers be allowed to analyze data?,362b19e9210ec9554c12f252f2be3c37,"[1, 1]"
2,741bff87-39dd-4579-a73e-940d6e100763,26-01-23-10-32,test_model,3,"{""role"": ""user"", ""content"": ""I disagree, I thi...",80ddef96ef70a3b1c4d6db9035a0c8c5,"[1, 1]"
3,741bff87-39dd-4579-a73e-940d6e100763,26-01-23-10-32,test_model,3,"{""role"": ""user"", ""content"": ""I understand wher...",03d468a705d55c21ae5127e6b7282726,"[3, 3]"
4,741bff87-39dd-4579-a73e-940d6e100763,26-01-23-10-32,test_model,3,"{""role"": ""user"", ""content"": ""User Giannis post...",5bc96cc6b4f326e4aee879868abddbc9,"[3, 3]"
5,910ccd35-c69b-4072-b7d9-c01c4e2ae7f7,26-01-23-10-32,test_model,3,Absolutely not,d8e011c97798456009a252a3782cf466,"[1, 1]"
6,910ccd35-c69b-4072-b7d9-c01c4e2ae7f7,26-01-23-10-32,test_model,3,Should programmers be allowed to analyze data?,d59bd11732c595a720742eefe1ccfb2c,"[1, 1]"
7,910ccd35-c69b-4072-b7d9-c01c4e2ae7f7,26-01-23-10-32,test_model,3,"{""role"": ""user"", ""content"": ""Are you kidding m...",14c87db7cf55a354119b5ff81ae6c0a6,"[3, 1]"
8,910ccd35-c69b-4072-b7d9-c01c4e2ae7f7,26-01-23-10-32,test_model,3,"{""role"": ""user"", ""content"": ""Emma, spare me th...",10e6d07e08d0451def3677a043a686bd,"[3, 3]"
9,910ccd35-c69b-4072-b7d9-c01c4e2ae7f7,26-01-23-10-32,test_model,3,"{""role"": ""user"", ""content"": ""Giannis, I may no...",895d0a290edddca6758247cdf2ecb897,"[3, 3]"
